In [3]:
import sys
sys.path.append('/home/user/PycharmProjects/reversi-game/')

In [5]:
import numpy as np

In [6]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

import stable_baselines3.common.callbacks as callbacks_module
from sb3_contrib.common.maskable.evaluation import evaluate_policy as masked_evaluate_policy

# Modify the namespace of EvalCallback directly
callbacks_module.evaluate_policy = masked_evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback

# from sb3_contrib.common.maskable.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
# from stable_baselines3.common.callbacks import EvalCallback

from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy, MaskableMultiInputActorCriticPolicy
from sb3_contrib.ppo_mask import MaskablePPO

from shutil import copyfile # keep track of generations
from collections import OrderedDict

from gymnasium.spaces import Discrete, Box, Dict, MultiDiscrete
from gymnasium.wrappers import FlattenObservation
import gymnasium.spaces as spaces
from game_logic import Othello
import numpy as np
import os, math
from itertools import cycle

In [7]:
import torch as th

In [8]:
# Settings
SEED = 19
NUM_TIMESTEPS = int(30_000_000)
EVAL_FREQ = int(10_000)
EVAL_EPISODES = int(100)
BEST_THRESHOLD = 0.2 # must achieve a mean score above this to replace prev best self

RENDER_MODE = False # set this to false if you plan on running for full 1000 trials.

LOGDIR = "models"
# LOGDIR = "delete_me"

In [56]:
class OthelloEnv(gym.Env):   
    def __init__(self):
        self.game = Othello()
        self.agent_turn = 1
        shape = self.game.board.shape  
        self.action_mapping = self.am(shape)
        self.action_space = Discrete(shape[0] * shape[1])  # sample - [x, y]
        self.observation_space = Dict({
                                        'board' : Box(0, 2, shape=shape, dtype=int),
                                        'chips' : MultiDiscrete([65, 65]),
                                        'player': Discrete(2, start=1)
                                      })        
        self.other_agent = None
        # self.reset_othello_gen = self.reset_othello()   
        
        self.episodes = 0    
        # self.global_reward = 0
    

    def am(self, shape):
        x, y = shape
        return [(n//x, n%x) for n in range(x * y)]
        

    # def reset_othello(self):
    #     '''resets game to starting position 
    #        and also changes starting player alternatively'''
    #     infinite_player_turn = cycle([1]*10 + [2]*10)
    #     while True:
    #         game = Othello()
    #         model_turn = next(infinite_player_turn)
    #         yield game, model_turn
    
    def change_to_latest_agent(self, agent):
        self.other_agent = agent

    def get_obs(self):        
        return OrderedDict({
            'board' : self.game.board,
            'chips' : np.array(self.game.chips),
            'player': self.game.player_turn
        })

    def get_chips_diff(self): #  from agent perspective
        idx = self.agent_turn - 1  # map [1, 2] to [0, 1]
        diff = self.game.chips[idx] - self.game.chips[1 - idx]
        return diff
        
    def check_game_ended(self):
        reward = 0
        done = False
        winner = self.game.get_winner()
        
        if winner is not None:
            self.episodes += 1
            if self.episodes % 10 == 0:
                print(f'Ep done - {self.episodes}.')
                # print(f'global_reward -- {self.global_reward}, -- won: {winner == self.agent_turn}')
            
            done = True
            if winner == self.agent_turn:
                # reward = max(abs(self.global_reward)*2, 1000)
                reward = 1
            elif winner == 3 - self.agent_turn: #  other agent turn/figure
                # reward = min(-abs(self.global_reward)*2, -1000)
                reward = -1
        return reward, done
    
    def render(self):  # todo 
        pass

    def close(self):  # todo
        pass

    def other_agent_play_move(self): 
        obs = self.get_obs()
        obs = spaces.flatten(self.observation_space, obs)#  need to flatten observation         
        action, _ = self.other_agent.predict(obs,
                                             action_masks=self.action_masks(),
                                             deterministic=False) 
        game_action = self.action_mapping[action]
        self.game.play_move(game_action)

    def step(self, action):
        # diff_chips_before = self.get_chips_diff()
        
        game_action = self.action_mapping[action]  
        self.game.play_move(game_action)

        # inner agent plays
        while self.game.get_winner() is None and self.game.player_turn != self.agent_turn: #  if game hasnt ended do moves if opponent doesnt have one 
            self.other_agent_play_move()

        # diff_chips_after = self.get_chips_diff()
        
        reward, done = self.check_game_ended()

        # turn = self.game.turn
        # if turn <= 58: #  not sure... feel like at the end theres no more moves to choose and high reward would be bad            
        #     factor = (turn // 10) + 1
        #     step_reward = factor * (diff_chips_after - diff_chips_before)
        #     self.global_reward += step_reward
        #     reward += factor * (diff_chips_after - diff_chips_before)

        
        info = {}
        truncated = False

                
        # Return step information
        return self.get_obs(), reward, done, truncated, info
    
    def reset(self, *args, **kwargs):
        # self.global_reward = 0
        self.game = Othello() # self.game, self.agent_turn = next(self.reset_othello_gen)
        if self.agent_turn == 2:
            self.other_agent_play_move()
        return self.get_obs(), None

    def action_masks(self):        
        valid_moves = self.game.valid_moves()
    
        mask = np.zeros(self.game.board.shape, dtype=bool)
        
        # Set True for each index in the set
        for index in valid_moves:
            mask[index] = True
        mask.flatten()
        return mask

In [57]:
class SelfPlayCallback(EvalCallback):
    # hacked it to only save new version offrom gymnasium.wrappers import FlattenObservation best model if beats prev self by BEST_THRESHOLD score
    # after saving model, resets the best score to be BEST_THRESHOLD
    def __init__(self, eval_env, *args, **kwargs):
        super().__init__(eval_env, *args, **kwargs)
        self.best_mean_reward = BEST_THRESHOLD
        self.generation = 0        
    def _on_step(self) -> bool:
        # result = super()._on_step() #  eval needs to be masked, its less efficient 
        result = super()._on_step()
        
        if result and self.best_mean_reward > BEST_THRESHOLD:
            self.generation += 1
            print("SELFPLAY: mean_reward achieved:", self.best_mean_reward)
            print("SELFPLAY: new best model, bumping up generation to", self.generation)            
            source_file = os.path.join(LOGDIR, "best_model.zip")
            backup_file = os.path.join(LOGDIR, "history_"+str(self.generation).zfill(8)+".zip")
            copyfile(source_file, backup_file)
            self.best_mean_reward = BEST_THRESHOLD
            agent = self.model.load(source_file)
                
            self.training_env.env_method("change_to_latest_agent", agent)           
            self.eval_env.env_method("change_to_latest_agent", agent) #  .env_method("method_name", args1, args2, kwargs1=kwargs1) 
        return result    

In [58]:
env = OthelloEnv()
env = Monitor(env=env)
env = FlattenObservation(env)

In [59]:
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[128, 64], vf=[64, 64]))
# Create the agent
#model = PPO("MlpPolicy", "CartPole-v1", policy_kwargs=policy_kwargs, verbose=1)

In [60]:
model = MaskablePPO(policy=MaskableActorCriticPolicy, 
                    env=env, 
                    verbose=1,
                    policy_kwargs=policy_kwargs#,
                    # learning_rate=1e-5, 
                    # n_steps=6144
                   )

Using cpu device
Wrapping the env in a DummyVecEnv.


In [61]:
starting_model_filepath = LOGDIR + '/start_model'
model.save(starting_model_filepath)

/home/user/venv/rl-env/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:283: UserWarning: Path 'models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [62]:
start_model_copy = model.load(starting_model_filepath)
env.unwrapped.change_to_latest_agent(start_model_copy)


env_eval = OthelloEnv()
env_eval = Monitor(env=env_eval)
env_eval = FlattenObservation(env_eval)

env_eval = DummyVecEnv(env_fns=[lambda: env_eval])
env_eval.envs[0].unwrapped.change_to_latest_agent(start_model_copy)

In [ ]:
eval_callback = SelfPlayCallback(    
    env_eval,
    best_model_save_path=LOGDIR,
    log_path=LOGDIR,
    eval_freq=EVAL_FREQ,
    n_eval_episodes=EVAL_EPISODES,
    deterministic=False 
    )


model.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

/home/user/venv/rl-env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


Ep done - 10.
Ep done - 20.
Ep done - 30.
Ep done - 40.
Ep done - 50.
Ep done - 60.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.3     |
|    ep_rew_mean     | 0.149    |
| time/              |          |
|    fps             | 282      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
Ep done - 70.
Ep done - 80.
Ep done - 90.
Ep done - 100.
Ep done - 110.
Ep done - 120.
Ep done - 130.
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30.1       |
|    ep_rew_mean          | 0.14       |
| time/                   |            |
|    fps                  | 234        |
|    iterations           | 2          |
|    time_elapsed         | 17         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01656268 |
|    clip_fraction     

Ep done - 620.
Ep done - 630.
Ep done - 640.
Ep done - 650.
Ep done - 660.
Ep done - 110.
Ep done - 120.
Ep done - 130.
Ep done - 140.
Ep done - 150.
Ep done - 160.
Ep done - 170.
Ep done - 180.
Ep done - 190.
Ep done - 200.
Eval num_timesteps=20000, episode_reward=0.13 +/- 0.98
Episode length: 30.20 +/- 0.57
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 30.2        |
|    mean_reward          | 0.13        |
| time/                   |             |
|    total_timesteps      | 20000       |
| train/                  |             |
|    approx_kl            | 0.017519198 |
|    clip_fraction        | 0.22        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.88       |
|    explained_variance   | 0.0211      |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00111    |
|    n_updates            | 90          |
|    policy_gradient_loss | -0.0573     |
|    value_loss  

/home/user/venv/rl-env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.change_to_latest_agent to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.change_to_latest_agent` for environment variables or `env.get_wrapper_attr('change_to_latest_agent')` that will search the reminding wrappers.
  logger.warn(


Ep done - 1000.
Ep done - 1010.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.2     |
|    ep_rew_mean     | 0.11     |
| time/              |          |
|    fps             | 140      |
|    iterations      | 15       |
|    time_elapsed    | 219      |
|    total_timesteps | 30720    |
---------------------------------
Ep done - 1020.
Ep done - 1030.
Ep done - 1040.
Ep done - 1050.
Ep done - 1060.
Ep done - 1070.
Ep done - 1080.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.1        |
|    ep_rew_mean          | -0.04       |
| time/                   |             |
|    fps                  | 137         |
|    iterations           | 16          |
|    time_elapsed         | 239         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.018902505 |
|    clip_fraction        | 0.237       |
|    clip_ra

...

New best mean reward!
SELFPLAY: mean_reward achieved: 0.79
SELFPLAY: new best model, bumping up generation to 109
Ep done - 39780.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.1     |
|    ep_rew_mean     | 0.78     |
| time/              |          |
|    fps             | 170      |
|    iterations      | 586      |
|    time_elapsed    | 7035     |
|    total_timesteps | 1200128  |
---------------------------------
Ep done - 39790.
Ep done - 39800.
Ep done - 39810.
Ep done - 39820.
Ep done - 39830.
Ep done - 39840.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.2        |
|    ep_rew_mean          | 0.87        |
| time/                   |             |
|    fps                  | 170         |
|    iterations           | 587         |
|    time_elapsed         | 7044        |
|    total_timesteps      | 1202176     |
| train/                  |             |
|    ap

Ep done - 40330.
Ep done - 40340.
Ep done - 40350.
Ep done - 40360.
Ep done - 40370.
Ep done - 40380.
Ep done - 40390.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.1        |
|    ep_rew_mean          | 0.55        |
| time/                   |             |
|    fps                  | 170         |
|    iterations           | 595         |
|    time_elapsed         | 7128        |
|    total_timesteps      | 1218560     |
| train/                  |             |
|    approx_kl            | 0.075213075 |
|    clip_fraction        | 0.126       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.178      |
|    explained_variance   | 0.464       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0644      |
|    n_updates            | 5940        |
|    policy_gradient_loss | -0.0249     |
|    value_loss           | 0.164       |
-----------------------------------------

Ep done - 40860.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.1        |
|    ep_rew_mean          | 0.66        |
| time/                   |             |
|    fps                  | 170         |
|    iterations           | 602         |
|    time_elapsed         | 7212        |
|    total_timesteps      | 1232896     |
| train/                  |             |
|    approx_kl            | 0.041216135 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.164      |
|    explained_variance   | 0.11        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0357      |
|    n_updates            | 6010        |
|    policy_gradient_loss | -0.0209     |
|    value_loss           | 0.112       |
-----------------------------------------
Ep done - 40870.
Ep done - 40880.
Ep done - 40890.
Ep done - 40900.
Ep done - 40910.
Ep done - 40920.

Ep done - 41350.
Ep done - 41360.
Ep done - 41370.
Ep done - 41380.
Ep done - 41390.
Ep done - 41400.
Ep done - 41410.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.2        |
|    ep_rew_mean          | 0.69        |
| time/                   |             |
|    fps                  | 171         |
|    iterations           | 610         |
|    time_elapsed         | 7296        |
|    total_timesteps      | 1249280     |
| train/                  |             |
|    approx_kl            | 0.040091306 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.154      |
|    explained_variance   | 0.339       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0225      |
|    n_updates            | 6090        |
|    policy_gradient_loss | -0.0176     |
|    value_loss           | 0.121       |
-----------------------------------------

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 30.2       |
|    ep_rew_mean          | 0.68       |
| time/                   |            |
|    fps                  | 171        |
|    iterations           | 617        |
|    time_elapsed         | 7380       |
|    total_timesteps      | 1263616    |
| train/                  |            |
|    approx_kl            | 0.04153005 |
|    clip_fraction        | 0.124      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.225     |
|    explained_variance   | 0.233      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0283     |
|    n_updates            | 6160       |
|    policy_gradient_loss | -0.0192    |
|    value_loss           | 0.119      |
----------------------------------------
Ep done - 41890.
Ep done - 41900.
Ep done - 41910.
Ep done - 41920.
Ep done - 41930.
Ep done - 41940.
Ep done - 41950.
--------------------

In [145]:
# start_model_copy.predict(env.reset()[0], action_masks=env.action_masks())
# model.predict(env.reset()[0], action_masks=env.action_masks())

In [134]:
env.reset()[0]

OrderedDict([('board',
              array([[0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 2, 0, 0, 0],
                     [0, 0, 0, 2, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0]])),
             ('player', 1)])

In [ ]:
obs = env.unwrapped.get_obs()
obs

In [131]:
new_obs = spaces.flatten(env.unwrapped.observation_space, obs)
new_obs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [133]:
for i in range(15):
    print(model.predict(new_obs))

(array(22), None)
(array(29), None)
(array(41), None)
(array(28), None)
(array(5), None)
(array(46), None)
(array(24), None)
(array(13), None)
(array(40), None)
(array(59), None)
(array(13), None)
(array(45), None)
(array(18), None)
(array(62), None)
(array(51), None)


In [115]:
new_obs

array([0, 2, 0, 1, 0, 0, 0, 0, 2, 0, 2, 2, 2, 1, 0, 1, 0, 2, 2, 2, 2, 2,
       1, 2, 1, 0, 2, 2, 1, 1, 0, 0, 0, 0, 1, 2, 2, 1, 1, 0, 0, 0, 1, 0,
       2, 2, 1, 1, 0, 2, 1, 0, 2, 2, 2, 1, 2, 0, 0, 0, 2, 0, 2, 0, 0, 1])

In [142]:
env_eval = OthelloEnv()
env_eval = Monitor(env=env_eval)
env_eval = FlattenObservation(env_eval)

env_eval = DummyVecEnv(env_fns=[lambda: env_eval])

In [147]:
model1 = MaskablePPO.load('ppo_masked_selfplay/history_00000385.zip')
model_random = MaskablePPO.load('ppo_masked_selfplay/history_00000170.zip')

In [148]:
env_eval.envs[0].unwrapped.change_to_latest_agent(model_random)

In [153]:
episode_rewards, episode_lengths = evaluate_policy(
                model1,
                env_eval,
                n_eval_episodes=100,                
                deterministic=True,
                return_episode_rewards=True,
                warn=True
            )

Ep done - 310.
Ep done - 320.
Ep done - 330.
Ep done - 340.
Ep done - 350.
Ep done - 360.
Ep done - 370.
Ep done - 380.
Ep done - 390.
Ep done - 400.


In [154]:
np.mean(episode_rewards)

-0.12